In [2]:
import os
from glob import glob

import main_fte
from py_utils import data_ops
import evaluate_fte
from lib import app
from py_utils import log

%load_ext autoreload
%autoreload 2

%matplotlib inline

# root_dir = os.path.join("..", "data")
root_dir = os.path.join("/Users/zico/OneDrive - University of Cape Town/CheetahReconstructionResults/cheetah_videos")
# If you are running the code remotely on the Linux i9.
# root_dir = os.path.join("/","data", "dlc", "to_analyse", "cheetah_videos")\
logger = log.logger("main")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Generate the pose functions if this has not been done before (note this should only be performed once). It will be saved in the root_dir.
functions = main_fte.create_pose_functions(root_dir)

In [7]:
# Configurations for the optimisation. 2017_08_29/top/phantom/run1_1 2017_08_29/top/phantom/flick1_1 2019_02_27/kiara/run
data_path = os.path.join("2017_08_29", "top", "jules", "run1_1")
start_frame = 20
end_frame = 120
dlc_thresh = 0.5

In [9]:
# Run the optimisation
main_fte.run(root_dir, data_path, start_frame, end_frame, dlc_thresh, init_ekf=False, single_view=0, generate_reprojection_videos=True, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

2021-06-21T22:42:23.507 | INFO | main_fte: Prepare data - Start
2021-06-21T22:42:24.505 | INFO | main_fte: Load H5 2D DLC prediction data
2021-06-21T22:42:24.941 | INFO | main_fte: Estimate the initial trajectory
2021-06-21T22:42:25.447 | INFO | main_fte: Prepare data - End
2021-06-21T22:42:25.451 | INFO | main_fte: Start frame: 19, End frame: 120, Frame rate: 90
2021-06-21T22:42:25.454 | INFO | main_fte: Setup optimisation - Start
2021-06-21T22:42:26.463 | INFO | main_fte: Measurement initialisation...Done
2021-06-21T22:42:33.910 | INFO | main_fte: Variable initialisation...Done
2021-06-21T22:42:51.673 | INFO | main_fte: Constaint initialisation...Done
2021-06-21T22:42:54.432 | INFO | main_fte: Objective initialisation...Done
2021-06-21T22:42:54.435 | INFO | main_fte: Setup optimisation - End
2021-06-21T22:42:54.438 | INFO | main_fte: Initialisation took 30.92s
2021-06-21T22:45:03.272 | INFO | main_fte: Optimisation solver took 128.83s
2021-06-21T22:45:03.274 | INFO | main_fte: Genera

In [ ]:
# Compare the optimisation with previous results i.e. compare the output pickle file fte.pickle. This is a visual inspection.abs
main_fte.plot_cheetah(root_dir, data_path, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

In [ ]:
# Compare the optimisation with previous results i.e. compare the output pickle file fte.pickle. This is a visual inspection.abs
main_fte.compare_cheetahs("/Users/zico/msc/dev/AcinoSet/data/2019_03_09/lily/run/fte_pw_1/fte.pickle", root_dir, data_path, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

In [ ]:
eval_dir = os.path.join("/Users/zico/msc/dev/AcinoSet/data", data_path, "fte_pw")
data = data_ops.load_pickle(os.path.join(eval_dir, "fte.pickle"))
results_dir = os.path.join(eval_dir, "evaluation")
os.makedirs(eval_dir, exist_ok=True)
max_delta_acc = evaluate_fte.eval_delta_acc(data, results_dir)
evaluate_fte.eval_model_error(data, results_dir)
evaluate_fte.eval_meas_error(data, results_dir)
logger.info(max_delta_acc)

In [ ]:
fig_fpath = os.path.join(os.path.join(os.path.join("..", "data"), data_path, "fte_pw"), 'fte.svg')
pw_file = data_ops.load_pickle(os.path.join(os.path.join("..", "data"), data_path, "fte_pw", "fte.pickle"))
app.plot_cheetah_states(pw_file["x"], out_fpath=fig_fpath)

In [ ]:
main_fte.plot_cost_functions()